In [9]:
# FEATURE SELECTION - MULTICLASS PROBLEM

In [10]:
# SCRIPT desorganizado (por organizar) apenas com a tentativa de melhorar a performance do problema multiclasse

In [11]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import make_scorer, accuracy_score, f1_score
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import GridSearchCV

In [12]:
fifty_mel_vs_keratosis_neither_features_targets = pd.read_csv('../dataset_versions/50neither_vs_mel_keratosis.csv')
fifty_mel_vs_keratosis_neither_features_targets.head()

,image_id,age_approximate,sex,melanoma,seborrheic_keratosis,neither,red_mode,green_mode,blue_mode,red_median,...,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
0,ISIC_0000001,30,female,0,0,1,87,53,46,116,...,4.441031,0.140831,2.777913,0.575211,2.318644,4.577623,0.021560,0.808480,0.006020,0.124740
1,ISIC_0000002,60,female,1,0,0,162,120,114,161,...,2.709728,0.168455,1.932127,0.540123,2.526498,0.231924,0.547760,0.007574,0.000012,0.941026
2,ISIC_0000004,80,male,1,0,0,192,132,171,192,...,18.402358,0.129868,4.275156,0.564438,2.312946,0.506435,0.043551,3.153283,0.021736,2.213424
3,ISIC_0000006,25,female,0,0,1,213,123,100,187,...,0.999912,0.641466,2.115149,0.592953,1.629161,14.107023,0.048779,0.049819,0.004353,0.046243
4,ISIC_0000009,30,female,0,0,1,187,122,90,194,...,1.866512,0.433151,2.553233,0.598177,1.871148,33.795948,0.021462,0.065349,0.016073,0.021467


In [13]:
fifty_mel_vs_keratosis_neither_features_targets = fifty_mel_vs_keratosis_neither_features_targets.drop(columns=["age_approximate", "sex"])

In [14]:
fifty_mel_vs_keratosis_neither_features_targets.columns[0]

'image_id'

In [15]:
# diagnostic column:
## 1 - melanoma, 2 - seborrheic keratosis, 3 - neither

for i,u in enumerate(fifty_mel_vs_keratosis_neither_features_targets['image_id']):
    if fifty_mel_vs_keratosis_neither_features_targets.loc[i,'melanoma'] == 0:
        if fifty_mel_vs_keratosis_neither_features_targets.loc[i,'seborrheic_keratosis'] == 1:
            fifty_mel_vs_keratosis_neither_features_targets.loc[i,'melanoma'] = 2
    else:
        continue
        
fifty_mel_vs_keratosis_neither_features_targets.drop(columns=['seborrheic_keratosis', 'neither'], inplace=True)
fifty_mel_vs_keratosis_neither_features_targets

,image_id,melanoma,red_mode,green_mode,blue_mode,red_median,green_median,blue_median,red_iqr,green_iqr,...,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
0,ISIC_0000001,0,87,53,46,116,72,55,86,58,...,4.441031,0.140831,2.777913,0.575211,2.318644,4.577623,0.021560,0.808480,0.006020,0.124740
1,ISIC_0000002,1,162,120,114,161,131,131,26,52,...,2.709728,0.168455,1.932127,0.540123,2.526498,0.231924,0.547760,0.007574,0.000012,0.941026
2,ISIC_0000004,1,192,132,171,192,128,157,36,44,...,18.402358,0.129868,4.275156,0.564438,2.312946,0.506435,0.043551,3.153283,0.021736,2.213424
3,ISIC_0000006,0,213,123,100,187,152,130,38,76,...,0.999912,0.641466,2.115149,0.592953,1.629161,14.107023,0.048779,0.049819,0.004353,0.046243
4,ISIC_0000009,0,187,122,90,194,138,100,35,60,...,1.866512,0.433151,2.553233,0.598177,1.871148,33.795948,0.021462,0.065349,0.016073,0.021467
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1309,ISIC_0015220,2,120,36,20,136,57,46,46,47,...,3.702858,0.155806,4.435975,0.188760,213.351940,11.504115,0.014350,0.271341,0.005850,0.070004
1310,ISIC_0015233,2,71,124,101,115,80,63,97,73,...,4.671226,0.112319,3.028206,0.536858,2.644843,11.205961,0.007821,0.237932,0.001880,0.042187
1311,ISIC_0015260,2,76,21,13,77,40,24,65,51,...,2.564094,0.197151,4.190389,0.181698,164.827636,8.735373,0.009154,0.904079,0.002138,0.132170
1312,ISIC_0015284,1,136,19,6,143,80,58,52,76,...,11.609996,0.069719,4.160087,0.526389,5.469382,2.553310,0.009927,2.045905,0.003094,0.333450


In [16]:
fifty_mel_vs_keratosis_neither_features_targets.columns.to_list()

['image_id',
 'melanoma',
 'red_mode',
 'green_mode',
 'blue_mode',
 'red_median',
 'green_median',
 'blue_median',
 'red_iqr',
 'green_iqr',
 'blue_iqr',
 'original_firstorder_10Percentile',
 'original_firstorder_90Percentile',
 'original_firstorder_Energy',
 'original_firstorder_Entropy',
 'original_firstorder_InterquartileRange',
 'original_firstorder_Kurtosis',
 'original_firstorder_Maximum',
 'original_firstorder_MeanAbsoluteDeviation',
 'original_firstorder_Mean',
 'original_firstorder_Median',
 'original_firstorder_Minimum',
 'original_firstorder_Range',
 'original_firstorder_RobustMeanAbsoluteDeviation',
 'original_firstorder_RootMeanSquared',
 'original_firstorder_Skewness',
 'original_firstorder_TotalEnergy',
 'original_firstorder_Uniformity',
 'original_firstorder_Variance',
 'original_glcm_Autocorrelation',
 'original_glcm_ClusterProminence',
 'original_glcm_ClusterShade',
 'original_glcm_ClusterTendency',
 'original_glcm_Contrast',
 'original_glcm_Correlation',
 'original_

In [17]:
# separate features and target
X = fifty_mel_vs_keratosis_neither_features_targets.drop(columns=["image_id", "melanoma"])
y = fifty_mel_vs_keratosis_neither_features_targets['melanoma']
#X_train, X_test, y_train, y_test = train_test_split(X, y)

In [18]:
X

,red_mode,green_mode,blue_mode,red_median,green_median,blue_median,red_iqr,green_iqr,blue_iqr,original_firstorder_10Percentile,...,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
0,87,53,46,116,72,55,86,58,40,112.0,...,4.441031,0.140831,2.777913,0.575211,2.318644,4.577623,0.021560,0.808480,0.006020,0.124740
1,162,120,114,161,131,131,26,52,67,157.0,...,2.709728,0.168455,1.932127,0.540123,2.526498,0.231924,0.547760,0.007574,0.000012,0.941026
2,192,132,171,192,128,157,36,44,55,26.0,...,18.402358,0.129868,4.275156,0.564438,2.312946,0.506435,0.043551,3.153283,0.021736,2.213424
3,213,123,100,187,152,130,38,76,101,138.0,...,0.999912,0.641466,2.115149,0.592953,1.629161,14.107023,0.048779,0.049819,0.004353,0.046243
4,187,122,90,194,138,100,35,60,58,144.0,...,1.866512,0.433151,2.553233,0.598177,1.871148,33.795948,0.021462,0.065349,0.016073,0.021467
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1309,120,36,20,136,57,46,46,47,51,97.0,...,3.702858,0.155806,4.435975,0.188760,213.351940,11.504115,0.014350,0.271341,0.005850,0.070004
1310,71,124,101,115,80,63,97,73,59,137.0,...,4.671226,0.112319,3.028206,0.536858,2.644843,11.205961,0.007821,0.237932,0.001880,0.042187
1311,76,21,13,77,40,24,65,51,34,121.0,...,2.564094,0.197151,4.190389,0.181698,164.827636,8.735373,0.009154,0.904079,0.002138,0.132170
1312,136,19,6,143,80,58,52,76,85,90.0,...,11.609996,0.069719,4.160087,0.526389,5.469382,2.553310,0.009927,2.045905,0.003094,0.333450


In [19]:
y

0       0
1       1
2       1
3       0
4       0
       ..
1309    2
1310    2
1311    2
1312    1
1313    2
Name: melanoma, Length: 1314, dtype: int64

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [21]:
X_train_std = StandardScaler().fit_transform(X_train)
X_test_std = StandardScaler().fit_transform(X_test)

y_train = y_train.values.reshape(-1, 1)
y_test = y_test.values.reshape(-1, 1)

In [22]:
rfe = RFE(estimator = LogisticRegression(max_iter=10000), n_features_to_select=5 , step=1)

_ = rfe.fit(X_train_std, y_train)

c:\users\utilizador\projects\aa_meb\aaenv\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\utilizador\projects\aa_meb\aaenv\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\utilizador\projects\aa_meb\aaenv\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\utilizador\projects\aa_meb\aaenv\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when 

c:\users\utilizador\projects\aa_meb\aaenv\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\utilizador\projects\aa_meb\aaenv\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\utilizador\projects\aa_meb\aaenv\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\utilizador\projects\aa_meb\aaenv\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when 

c:\users\utilizador\projects\aa_meb\aaenv\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\utilizador\projects\aa_meb\aaenv\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\utilizador\projects\aa_meb\aaenv\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\utilizador\projects\aa_meb\aaenv\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when 

c:\users\utilizador\projects\aa_meb\aaenv\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\utilizador\projects\aa_meb\aaenv\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\utilizador\projects\aa_meb\aaenv\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [23]:
X_train.loc[:, rfe.support_]

,blue_iqr,original_glcm_ClusterTendency,original_glcm_DifferenceEntropy,original_glcm_InverseVariance,original_gldm_GrayLevelVariance
459,65,0.130961,0.248526,0.042323,0.031119
1046,9,3.888905,0.250968,0.047638,1.243982
820,28,0.749675,0.134231,0.019548,0.206567
711,21,0.095303,0.076708,0.009745,0.018733
649,13,0.316302,0.383186,0.076916,0.111004
...,...,...,...,...,...
763,53,0.218619,0.356991,0.067968,0.075541
835,60,0.709233,0.592219,0.144063,0.210319
1216,24,0.370091,0.254437,0.041270,0.096805
559,77,0.593041,0.649661,0.166933,0.185241


In [25]:
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score

In [ ]:
# ESTE FOI A MELHOR ACCURACY CONSEGUIDA, O RESTO FORAM OUTROS TESTES

In [30]:
X_train_sel = rfe.transform(X_train_std)
X_test_sel = rfe.transform(X_test_std)

In [31]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV

In [32]:
# tunning Log_Reg
model = LogisticRegression()

# define evaluation
cv = RepeatedStratifiedKFold(n_splits=2, n_repeats=3, random_state=1)

# define search space
space = dict()
space['solver'] = ['newton-cg', 'lbfgs', 'liblinear']
space['penalty'] = ['none', 'l1', 'l2', 'elasticnet']
space['C'] = [1e-3, 1e-2, 1e-1, 1, 10, 100]

# define search
search = GridSearchCV(model, space, scoring='accuracy', n_jobs=-1, cv=cv)

# execute search
result = search.fit(X_train_sel, y_train)

c:\users\utilizador\projects\aa_meb\aaenv\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
216 fits failed out of a total of 432.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
36 fits failed with the following error:
Traceback (most recent call last):
  File "c:\users\utilizador\projects\aa_meb\aaenv\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\users\utilizador\projects\aa_meb\aaenv\lib\site-packages\sklearn\linear_model\_logistic.py", line 1091, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "c:\users\utilizador\projects\aa_meb\aaenv\lib\site-packages\sklearn\linear_mode

In [33]:
print('Best score reached: {} with params: {} '.format(result.best_score_, result.best_params_))

Best score reached: 0.5671783010933557 with params: {'C': 0.001, 'penalty': 'none', 'solver': 'newton-cg'} 


In [42]:
y_predF = result.predict(X_test_sel)

In [47]:
print('Model accuracy : {0:0.4f}'. format(accuracy_score(y_test, y_predF)))
print('Model recall : {0:0.4f}'. format(recall_score(y_test, y_predF, average='weighted') ))
print('Model roc/auc : {0:0.4f}'. format(roc_auc_score(y_test, result.predict_proba(X_test_sel), multi_class='ovr', 
                                                       average="macro")))

Model accuracy : 0.5684
Model recall : 0.5684
Model roc/auc : 0.6942


In [96]:
#### XGBOOST

# tunning Log_Reg
xgb = XGBClassifier()

xgb_params = {"n_estimators": [50, 100, 150],
              "subsample": [0.5, 0.8, 1],
              "max_depth": [1,3,4],
              "learning_rate": [0.1, 0.01, 0.01]}

xgb_cv = GridSearchCV(xgb, xgb_params, cv=5, n_jobs=-1, verbose=2).fit(X_train, y_train)

# execute search
result = xgb_cv.fit(X_train_sel, y_train)

Fitting 5 folds for each of 81 candidates, totalling 405 fits
Fitting 5 folds for each of 81 candidates, totalling 405 fits


In [97]:
xgb_cv.best_params_

{'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 50, 'subsample': 0.8}

In [99]:
y_predF = result.predict(X_test_sel)

In [100]:
print('Model accuracy : {0:0.4f}'. format(accuracy_score(y_test, y_predF)))

Model accuracy : 0.5805


In [ ]:
### SVC

In [103]:
model_list = [SVC, LinearSVC]

svc_params = [
                   {"gamma": "scale"}, {"gamma":"auto"},
                   {"kernel":'linear'}, {"kernel": 'poly'}, {"kernel": "rbf"}, {"kernel": "sigmoid"}, 
                   {"C":0.01}, {"C":0.1}, {"C":1}, {"C":5}, {"C":10}
             ]

## linearSVC
lin_svc_params = [
                  {"C":0.01}, {"C":0.1}, {"C":1}, {"C":5}, {"C":10}
                 ]


for model in model_list:
    print(model)

<class 'sklearn.svm._classes.SVC'>
<class 'sklearn.svm._classes.LinearSVC'>


In [104]:
### create list with models and params
modelclasses = [
    ["svc", SVC, svc_params],
    ["linearSVC", LinearSVC, lin_svc_params],
]

In [105]:
### iterate through each model and parameters, evaluate and get scores
insights = []
for modelname, Model, params_list in modelclasses:
    for params in params_list:
        model = Model(**params)
        model.fit(X_train_sel, y_train)
        score = model.score(X_test_sel, y_test)
        
        insights.append((modelname, model, params, score))

c:\users\utilizador\projects\aa_meb\aaenv\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\utilizador\projects\aa_meb\aaenv\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\utilizador\projects\aa_meb\aaenv\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\utilizador\projects\aa_meb\aaenv\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when 

In [106]:
model_data = pd.DataFrame(columns=['model','params','accuracy'])
for modelname, model, params, score in insights: 
    model_data = model_data.append(pd.Series([modelname, params, score], index=model_data.columns), ignore_index=True)

C:\Users\Utilizador\AppData\Local\Temp\ipykernel_112108\4031908477.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  model_data = model_data.append(pd.Series([modelname, params, score], index=model_data.columns), ignore_index=True)


In [107]:
model_data

,model,params,accuracy
0,svc,{'gamma': 'scale'},0.589666
1,svc,{'gamma': 'auto'},0.589666
2,svc,{'kernel': 'linear'},0.620061
3,svc,{'kernel': 'poly'},0.568389
4,svc,{'kernel': 'rbf'},0.589666
5,svc,{'kernel': 'sigmoid'},0.428571
6,svc,{'C': 0.01},0.525836
7,svc,{'C': 0.1},0.525836
8,svc,{'C': 1},0.589666
9,svc,{'C': 5},0.589666


---

In [57]:
# import modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.pipeline import make_pipeline
from lazypredict.Supervised import LazyClassifier

from xgboost import XGBClassifier
from sklearn.metrics import classification_report


from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import make_scorer, accuracy_score, f1_score
from sklearn.model_selection import GridSearchCV

In [33]:
# load dataset
fifty_mel_vs_keratosis_neither_features_targets = pd.read_csv('../fifty_neither_vs_mel_keratosis_features_targets.csv')
fifty_mel_vs_keratosis_neither_features_targets.head() 

,image_id,green_iqr,blue_iqr,original_firstorder_Skewness,original_glcm_Autocorrelation,original_glcm_Imc1,original_glcm_Imc2,original_gldm_LargeDependenceEmphasis,original_gldm_LargeDependenceLowGrayLevelEmphasis,original_glrlm_LongRunLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_ngtdm_Complexity,original_ngtdm_Strength,melanoma,seborrheic_keratosis,neither
0,ISIC_0000001,58,40,-0.49,6.02,-0.40,0.77,4.67,0.97,0.30,2.78,0.81,0.12,0,0,1
1,ISIC_0000002,52,67,-0.31,3.99,-0.56,0.25,5.05,1.28,0.38,1.93,0.01,0.94,1,0,0
2,ISIC_0000003,57,48,-0.04,3.86,-0.38,0.43,5.16,1.41,0.43,2.21,0.12,0.08,0,0,1
3,ISIC_0000004,44,55,0.59,31.01,-0.77,0.99,4.67,0.81,0.27,4.28,3.15,2.21,1,0,0
4,ISIC_0000007,70,83,-0.75,3.93,-0.45,0.36,4.84,1.26,0.40,1.89,0.04,0.42,0,0,1


In [34]:
# diagnostic column:
## 1 - melanoma, 2 - seborrheic keratosis, 3 - neither

for i,u in enumerate(fifty_mel_vs_keratosis_neither_features_targets['image_id']):
    if fifty_mel_vs_keratosis_neither_features_targets.loc[i,'melanoma'] == 0:
        if fifty_mel_vs_keratosis_neither_features_targets.loc[i,'seborrheic_keratosis'] == 1:
            fifty_mel_vs_keratosis_neither_features_targets.loc[i,'melanoma'] = 2
    else:
        continue
        
fifty_mel_vs_keratosis_neither_features_targets.drop(columns=['seborrheic_keratosis', 'neither'], inplace=True)
fifty_mel_vs_keratosis_neither_features_targets

,image_id,green_iqr,blue_iqr,original_firstorder_Skewness,original_glcm_Autocorrelation,original_glcm_Imc1,original_glcm_Imc2,original_gldm_LargeDependenceEmphasis,original_gldm_LargeDependenceLowGrayLevelEmphasis,original_glrlm_LongRunLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_ngtdm_Complexity,original_ngtdm_Strength,melanoma
0,ISIC_0000001,58,40,-0.49,6.02,-0.40,0.77,4.67,0.97,0.30,2.78,0.81,0.12,0
1,ISIC_0000002,52,67,-0.31,3.99,-0.56,0.25,5.05,1.28,0.38,1.93,0.01,0.94,1
2,ISIC_0000003,57,48,-0.04,3.86,-0.38,0.43,5.16,1.41,0.43,2.21,0.12,0.08,0
3,ISIC_0000004,44,55,0.59,31.01,-0.77,0.99,4.67,0.81,0.27,4.28,3.15,2.21,1
4,ISIC_0000007,70,83,-0.75,3.93,-0.45,0.36,4.84,1.26,0.40,1.89,0.04,0.42,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1309,ISIC_0015220,47,51,0.34,6.16,-0.80,0.96,7.48,2.28,5.15,4.44,0.27,0.07,2
1310,ISIC_0015233,73,59,0.00,7.06,-0.75,0.91,5.03,0.86,0.27,3.03,0.24,0.04,2
1311,ISIC_0015260,51,34,0.55,4.31,-0.63,0.89,7.51,2.81,5.59,4.19,0.90,0.13,2
1312,ISIC_0015284,76,85,-0.06,16.20,-0.81,0.99,5.21,0.73,0.55,4.16,2.05,0.33,1


In [35]:
# separate features and target
X = fifty_mel_vs_keratosis_neither_features_targets.iloc[:,1:-2]
y = fifty_mel_vs_keratosis_neither_features_targets['melanoma']
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [43]:
xgb = XGBClassifier()

In [44]:
xgb_params = {"n_estimators": [50, 100, 150],
              "subsample": [0.5, 0.8, 1],
              "max_depth": [1,3,4],
              "learning_rate": [0.1, 0.01, 0.01]}

xgb_cv = GridSearchCV(xgb, xgb_params, cv=5, n_jobs=-1, verbose=2).fit(X_train, y_train)

Fitting 5 folds for each of 81 candidates, totalling 405 fits


In [45]:
xgb_cv.best_params_

{'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 50, 'subsample': 0.5}

In [46]:
xgb_tuned = XGBClassifier(learning_rate=0.1,
                              max_depth=3,
                              n_estimators=100,
                              subsample=0.8).fit(X_train, y_train)

In [47]:
y_pred = xgb_tuned.predict(X_test)

In [48]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.62      0.78      0.69       168
           1       0.48      0.30      0.37        91
           2       0.62      0.54      0.58        70

    accuracy                           0.60       329
   macro avg       0.57      0.54      0.55       329
weighted avg       0.58      0.60      0.58       329



In [51]:
# Build the MPL
mlp_mk = MLPClassifier(max_iter=500)


# Hyperparameters to test
params = {
    'activation': ['tanh', 'relu','identity','logistic'],
    'solver': ['sgd', 'adam','sgd'],
    'alpha': [0.0001, 0.001, 0.05],
    'learning_rate': ['constant','adaptive', 'invscaling'],
}

# scale data
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


# train and evaluate the models
neural_net = GridSearchCV(mlp_mk, params, n_jobs=-1, cv=3)
neural_net.fit(X_train, y_train) 
print("Best parameter (CV score=%0.3f):" % neural_net.best_score_)
print(neural_net.best_params_)


Best parameter (CV score=0.584):
{'activation': 'relu', 'alpha': 0.0001, 'learning_rate': 'constant', 'solver': 'sgd'}


In [52]:
# test the model on new data                
y_true, y_pred = y_test , neural_net.predict(X_test)
print('Results on the test set: ')
print(classification_report(y_true, y_pred))

Results on the test set: 
              precision    recall  f1-score   support

           0       0.60      0.83      0.69       168
           1       0.47      0.16      0.24        91
           2       0.63      0.56      0.59        70

    accuracy                           0.59       329
   macro avg       0.56      0.52      0.51       329
weighted avg       0.57      0.59      0.55       329



In [64]:
### define list of models to train and evaluate
model_list = [SVC, LinearSVC]

In [65]:
### define parameters for each model 
## svc
svc_params = [
                   {"gamma": "scale"}, {"gamma":"auto"},
                   {"kernel":'linear'}, {"kernel": 'poly'}, {"kernel": "rbf"}, {"kernel": "sigmoid"}, 
                   {"C":0.01}, {"C":0.1}, {"C":1}, {"C":5}, {"C":10}
             ]

## linearSVC
lin_svc_params = [
                  {"C":0.01}, {"C":0.1}, {"C":1}, {"C":5}, {"C":10}
                 ]


for model in model_list:
    print(model)

<class 'sklearn.svm._classes.SVC'>
<class 'sklearn.svm._classes.LinearSVC'>


In [66]:
### create list with models and params
modelclasses = [
    ["svc", SVC, svc_params],
    ["linearSVC", LinearSVC, lin_svc_params],
]

In [67]:
### iterate through each model and parameters, evaluate and get scores
insights = []
for modelname, Model, params_list in modelclasses:
    for params in params_list:
        model = Model(**params)
        model.fit(X_train, y_train)
        score = model.score(X_test, y_test)
        
        insights.append((modelname, model, params, score))

In [68]:
model_data = pd.DataFrame(columns=['model','params','accuracy'])
for modelname, model, params, score in insights: 
    model_data = model_data.append(pd.Series([modelname, params, score], index=model_data.columns), ignore_index=True)

In [69]:
model_data

,model,params,accuracy
0,svc,{'gamma': 'scale'},0.58
1,svc,{'gamma': 'auto'},0.58
2,svc,{'kernel': 'linear'},0.58
3,svc,{'kernel': 'poly'},0.54
4,svc,{'kernel': 'rbf'},0.58
5,svc,{'kernel': 'sigmoid'},0.48
6,svc,{'C': 0.01},0.51
7,svc,{'C': 0.1},0.51
8,svc,{'C': 1},0.58
9,svc,{'C': 5},0.57


In [70]:
# ADABOOST with SCV as base estimator

In [71]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC

In [77]:
parameters = {
    'n_estimators': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 20],
    'learning_rate': [(0.97 + x / 100) for x in range(0, 8)],
    'algorithm': ['SAMME', 'SAMME.R']
}

ab_clf = AdaBoostClassifier(random_state=42)

clf = GridSearchCV(ab_clf, parameters, cv=5, n_jobs=-1).fit(X_train, y_train)

y_pred = model.predict(X_test)

In [74]:
svc=SVC(probability=True, kernel='rbf')

# Create adaboost classifer object
abc =AdaBoostClassifier(n_estimators=50, base_estimator=svc,learning_rate=1)

# Train Adaboost Classifer
model = abc.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = model.predict(X_test)

In [78]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.51      1.00      0.68       168
           1       0.00      0.00      0.00        91
           2       0.00      0.00      0.00        70

    accuracy                           0.51       329
   macro avg       0.17      0.33      0.23       329
weighted avg       0.26      0.51      0.35       329

